<a href="https://colab.research.google.com/github/aghapygad336/Fake-and-real-news/blob/master/Fake_and_real_news_dataset_Classifying_the_news.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from collections import Counter
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
from keras.layers.normalization import BatchNormalization


In [ ]:
!pip install pyspark==2.4.5
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))
spark = SparkSession \
    .builder \
    .getOrCreate()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
print(stopwords.words('english'))

In [ ]:
import nltk
nltk.download('punkt')


In [ ]:
def cleanDF(df):
    df.drop(['date', 'title'], axis=1, inplace=True)
    df.text = df.text.str.lower()
    df.text = df.text.str.replace(r'http[\w:/\.]+','<URL>') # remove urls
    df.text = df.text.str.replace(r'[^\.\w\s]','') #remove everything but characters and punctuation
    df.text = df.text.str.replace(r'\.\.+','.') #replace multple periods with a single one
    df.text = df.text.str.replace(r'\.',' . ') #replace multple periods with a single one
    df.text = df.text.str.replace(r'\s\s+',' ') #replace multple white space with a single one
    df.text = df.text.str.strip() 
    print(df.shape)
    df.head()
    return df


In [ ]:
True_df = pd.read_csv('/content/drive/My Drive/ask/FakeNews/True.csv')
True_df.head()


In [ ]:
True_df=cleanDF(True_df)
True_df.head()


In [ ]:
Fake_df = pd.read_csv('/content/drive/My Drive/ask/FakeNews/Fake.csv')
Fake_df.head()

In [ ]:
Fake_df=cleanDF(Fake_df)
Fake_df.head()


In [ ]:
Fake_df['CLASS'] = 0
Fake_df

In [ ]:
True_df['CLASS'] = 1
True_df

In [ ]:
frames = [Fake_df,True_df]

Data_df = pd.concat(frames)

In [ ]:
Data_df

Vectorization -- Word2Vec
Word2Vec is one of the most popular technique to learn word embeddings using shallow neural network. It was developed by Tomas Mikolov in 2013 at Google.

Word embedding is the most popular representation of document vocabulary. It is capable of capturing context of a word in a document, semantic and syntactic similarity, relation with other words, etc.



In [ ]:
y = Data_df["CLASS"].values
#Converting X to format acceptable by gensim, removing annd punctuation stopwords in the process
X = []
stop_words = set(nltk.corpus.stopwords.words("english"))
tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
for par in Data_df["text"].values:
    tmp = []
    sentences = nltk.sent_tokenize(par)
    for sent in sentences:
        sent = sent.lower()
        tokens = tokenizer.tokenize(sent)
        filtered_words = [w.strip() for w in tokens if w not in stop_words and len(w) > 1]
        tmp.extend(filtered_words)
    X.append(tmp)

del Data_df


In [ ]:
y.shape

In [ ]:
import gensim
#Dimension of vectors we are generating
EMBEDDING_DIM = 100

#Creating Word Vectors by Word2Vec Method (takes time...)
w2v_model = gensim.models.Word2Vec(sentences=X, size=EMBEDDING_DIM, window=5, min_count=1)

In [ ]:
len(w2v_model.wv.vocab)


In [ ]:
# Tokenizing Text -> Repsesenting each word by a number
# Mapping of orginal word to number is preserved in word_index property of tokenizer

#Tokenized applies basic processing like changing it yo lower case, explicitely setting that as False
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)

X = tokenizer.texts_to_sequences(X)


In [ ]:
#Lets keep all news to 700, add padding to news with less than 700 words and truncating long ones
maxlen = 700 
#Making all news of size maxlen defined above
X = pad_sequences(X, maxlen=maxlen)


In [ ]:
vocab_size = len(tokenizer.word_index) + 1


In [ ]:
#Lets check few word to numerical replesentation
#Mapping is preserved in dictionary -> word_index property of instance
word_index = tokenizer.word_index
for word, num in word_index.items():
    print(f"{word} -> {num}")
    if num == 10:
        break  

In [ ]:
# Function to create weight matrix from word2vec gensim model
def get_weight_matrix(model, vocab):
    # total vocabulary size plus 0 for unknown words
    vocab_size = len(vocab) + 1
    # define weight matrix dimensions with all 0
    weight_matrix = np.zeros((vocab_size, EMBEDDING_DIM))
    # step vocab, store vectors using the Tokenizer's integer mapping
    for word, i in vocab.items():
        weight_matrix[i] = model[word]
    return weight_matrix
embedding_vectors = get_weight_matrix(w2v_model, word_index)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y) 


In [ ]:
print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)


In [ ]:
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
MAX_NB_WORDS = 60000
EMBEDDING_DIM = 5000



In [ ]:
from keras.regularizers import l2
from keras import regularizers


In [ ]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
#Defining Neural Network
#Non-trainable embeddidng layer
#LSTM 
model.add(LSTM(units=128 , return_sequences = True , recurrent_dropout = 0.25 , dropout = 0.25))
model.add(LSTM(units=64 , recurrent_dropout = 0.2 , dropout = 0.5))
model.add(Dense(units = 32 , activation = 'relu'))
model.add(Dense(1, activation='sigmoid'))
model.layers[0].trainable = False
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history_lstm =model.fit(X_train, y_train, epochs=20 ,batch_size=128, workers=10, validation_data=(X_test, y_test))

$ANALYSIS AFTER TRAINING OF MODEL$


In [ ]:
print("Accuracy of the model on Training Data is - " , model.evaluate(X_train,y_train)[1]*100)
print("Accuracy of the model on Testing Data is - " , model.evaluate(X_test,y_test)[1]*100)


In [ ]:
plt.plot(history_lstm.history['loss'])
plt.plot(history_lstm.history['val_loss'])
plt.title("Train vs Validation Loss")
plt.ylabel('loss')
plt.xlabel('epoch')
plt.grid(False)
plt.legend(['train', 'test'], loc='upper right')

In [ ]:
import seaborn as sns

pred = model.predict_classes(X_test)
cm = confusion_matrix(y_test,pred)
cm = pd.DataFrame(cm , index = ['Fake','Real'] , columns = ['Fake','Real'])
plt.figure(figsize = (10,10))
sns.heatmap(cm,cmap= "Blues", linecolor = 'black' , linewidth = 1 , annot = True, fmt='' , xticklabels = ['Fake','Real'] , yticklabels = ['Fake','Real'])


